In [1]:
import pandas as pd 
import numpy as np 
from math import exp
import matplotlib
import matplotlib.pyplot as plt

Функция адаптации

Нужна для того, чтобы привести все показатели к одному виду - такому, при котором степень их желательности увеличивается вместе с увеличением их значений. Реализовано следующим образом: в выбранных колонках (в тех, в которых степень желательности растет вместе с уменьшением значений) максимальное значение приравнивается к нулю, после чего из максимального значения вычитаются остальные.

In [2]:
def adaptation(table, list):
    alternatives = table.values
    maximum = np.amax(alternatives, axis = 0)

    for i in range(len(list)):
        k = list[i]
        category = alternatives[:,k]
        for j in range(len(category)):
            if category[j] < maximum[k]:
                category[j] = maximum[k] - category[j]
            else:
                category[j] = category[j] - maximum[k]
        alternatives[:,k] = category

    return alternatives

Функция масштабирования в диапазоне

Нужна для того, чтобы можно было сравнивать данные различных размерностей. Реализовано с помощью формулы масштабирования в диапазоне, в котором a - минимальное значение желаемого диапазона, b - максимальное. 

In [3]:
def scale(alternatives, a, b):
    scaling = alternatives.astype(float)

    maxval = np.amax(alternatives, axis = 0)
    minval = np.amin(alternatives, axis = 0)
    for i in range(len(alternatives[1,:])):
        category = alternatives[:,i]
        for j in range(len(category)):
            category[j] = ((((category[j] - minval[i]) * (b - a))) / (maxval[i] - minval[i])) + a
        scaling[:,i] = category
    
    x = np.arange(0, len(scaling[:,1]))
    fig, ax = plt.subplots()
    
    ax.bar(x, scaling[:,-1], color='#00FFF5')
    plt.title('scale')
    plt.show()

    return scaling

Функция желательности Макмиллана

Для нее как раз и требовалось масштабирование в диапазоне. Реализовано в соответствии с формулой данной функции желательности.

In [4]:
def macmillan(scaling):
    macmill = scaling.astype(float)
    for i in range(len(macmill[1,:])):
        category = macmill[:,i]
        for j in range(len(category)):
            k = category[j]
            category[j] = (exp(-exp(-k)) - exp(-1)) / (1 - exp(-1))
            if category[j] < 0:
                category[j] = 0
        
        macmill[:,i] = category
    
    x = np.arange(0, len(macmill[:,1]))
    fig, ax = plt.subplots()
    
    ax.bar(x, macmill[:,-1], color='#FF00EB')
    plt.title('MC')
    plt.show()

    return macmill

Функция желательности Дерринджера

Для нее масштабирование не требуется, оно происходит в процессе расчета самой функции. Реализовано в соответствии с формулой функции желательности Дерринджера, в качестве значений r будет взято три значения от 2 до 4.

In [5]:
def derringer(relevant, r):
    derr = relevant.astype(float)
    print(derr)

    maxval = np.amax(derr, axis = 0)
    minval = np.amin(derr, axis = 0) 
    for i in range(len(maxval)):
        minval[i] = minval[i] + ((maxval[i] - minval[i]) * 0.15)

    
    for i in range(len(derr[1,:])):
        category = derr[:,i]
        for j in range(len(category)):
            if category[j] <= minval[i]:
                category[j] = 0
            elif category[j] < maxval[i]:
                category[j] = ((category[j] - minval[i]) / (maxval[i] - minval[i])) ** r
            else:
                category[j] = 1
        
        derr[:,i] = category

    return derr

Функция вычисления среднего значения функции желательности Дерринджера

In [6]:
def derrMean(r2, r3, r4):
    meanList = []
    for i in range(len(r2[:,1])):
        meanCategory = []
        for j in range(len(r2[1,:])):
            meanval = (r2[i, j] + r3[i, j] + r4[i,j]) / 3
            meanCategory.append(meanval)
        meanList.append(meanCategory)
    
    means = np.array(meanList)

    x = np.arange(0, len(means[:,1]))
    y = means[:,-1]
    plt.plot(x, y, color='#F6FF00')
    plt.xticks(range(0, len(means[:,1])))
    plt.title('DR')
    plt.show()

    return(means)

Функция расчета коэффициентов категорий с помощью метода анализа иерархий

Необходимо для дальнейшей свертки признаков по взвешенному среднему. Приоритетность категорий определяет лицо, принимающее решение.

In [7]:
def weight(list):
    i = 0
    contrasts = []
    while i < (len(list)):
        current = list[i]
        elem = []
        for j in range(len(list)):
            contrast = current / list[j]
            elem.append(contrast)
                
        contrasts.append(elem)
        i += 1
    
    contrastArr = np.array(contrasts)
    total = contrastArr.sum(axis = 0)

    i = 0
    adj = []
    while i < (len(list)):
        elem = []
        for j in range(len(contrastArr[i])):
            num = contrastArr[i, j] / total[j]
            elem.append(num)
        
        adj.append(elem)
        i += 1

    adjArr = np.array(adj)
    weightes = []
    for i in range(len(list)):
        roow = adjArr[i]
        num = np.mean(roow)
        weightes.append(num)
    
    weightes = np.array(weightes)

    return weightes 

Функция свертки

Нужна для выбора лучшей альтернативы из представленных в классе. 

In [8]:
def roll(wantFunc, weightes):
    rolled = []
    for i in range(len(wantFunc[:,1])):
        rolling = []
        for j in range(len(weightes)):
            num = wantFunc[i][j] * weightes[j]
            rolling.append(num)

        ocb = sum(rolling)
        rolled.append(ocb)

    rolled = np.array(rolled)

    x = np.arange(0, len(rolled))
    fig, ax = plt.subplots()
    
    ax.bar(x, rolled, color='#78FF00')
    plt.title('rolled')
    plt.show()

    return rolled

Функция выбора лучшей альтернативы из представленных в классе 

Возвращает номер строки той альтернативы, которая является наиболее желательной в данном классе.

In [9]:
def chooseBest(mcRolled, drRolled, weights, matchTable):
    mcBest = mcRolled.argmax()
    drBest = drRolled.argmax()

    if mcBest == drBest:
        best = mcBest
    else:
        drChoose = []
        mcChoose = []
        for i in range(len(matchTable[1,:])):
            mcNum = matchTable[mcBest, i] * weights[i]
            drNum = matchTable[drBest, i] * weights[i]

            drChoose.append(drNum)
            mcChoose.append(mcNum)
        
        drSum = sum(drChoose)
        mcSum = sum(mcChoose)

        if drSum > mcSum:
            best = drBest
        else: 
            best = mcBest

    return best

Финальная функция, которая использует в себе все написанные выше функции

Результатом ее работы является набор значений лучшей альтернативы в каждом классе.

In [10]:
def bestInClass(path, colsToTransform, importance):

    table = pd.read_excel(path, index_col = 0)
    adapted = adaptation(table, colsToTransform)
    scaled = scale(adapted.copy(), -5, 15)
    mcReady = macmillan(scaled)

    rTwo = derringer(adapted.copy(), 2)
    rThree= derringer(adapted.copy(), 3)
    rFour = derringer(adapted.copy(), 4)

    drReady = derrMean(rTwo, rThree, rFour)
    weightsReady = weight(importance)

    rollMc = roll(mcReady, weightsReady.copy())
    rollDr = roll(drReady, weightsReady.copy())
    
    numOfBest = chooseBest(rollMc, rollDr, weightsReady.copy(), adapted.copy())

    bestie = table.iloc[numOfBest,:].values

    print(bestie)
    return bestie

Стоит отметить, что все происходящее далее может варьироваться в зависимости от задачи. 

В данном случае задача звучит следующим образом: компания планирует расширять свою деятельность. Расширение заключается в выходе на новый регион для сбыта своей продукции. Помимо расходов на маркетинг, организацию деятельности компании и т.д., необходимо принять решение о выборе стратегии выхода на новый рынок. Под стратегией подразумевается способ обслуживания спроса, который предполагается в этом регионе. Для решения данной задачи предлагается четыре варианта: постройка своего собственного склада, аренда склада, перевозки с помощью транспортной компании и использование услуг логистического посредника. В каждом варианте есть 10 альтернатив - 10 разных предложений, которые компания может рассматривать.

Решение необходимо принимать на основании финансовых показателей. 

Таким образом, необходимо сначала определить наиболее выгодную альтернативу в каждом классе, после чего необходимо будет посчитать расходы за пять лет по каждому классу. Выбранная стратегия будет соотвествовать классу с наименьшими издержками.

In [ ]:
arenda = bestInClass('Arenda.xlsx', [0, 3, 4], [4, 3, 2, 1, 5])
posrednik = bestInClass('Posrednik.xlsx', [0, 2], [2, 1, 3])
sklad = bestInClass('Sklad.xlsx', [0, 2, 3], [2, 1, 3, 4])
tc = bestInClass('TC.xlsx', [0, 2, 3], [3, 1, 2, 4])

In [ ]:
volume = 67400 #объем перевозок
extra = 0.05 #% объема экстренных поставок

extraVolume = volume * extra
basicVolume = volume - extraVolume

In [ ]:
finArenda = [arenda[0] * 5, arenda[3] * 5, arenda[4] * 5]
finPosrednik = [posrednik[0] * 5, posrednik[2] * 5]
finSklad = [sklad[0], sklad[2], sklad[3] * 5]
finTc = [tc[0] * basicVolume * 5, tc[2] * extraVolume * 5, tc[3] * 5 ]

costsList = [sum(finArenda), sum(finPosrednik), sum(finSklad), sum(finTc)]

theBest = min(costsList)
bestIndex = costsList.index(theBest)

In [ ]:
if bestIndex == 0:
    name = 'Arenda'
elif bestIndex == 1:
    name = 'Posrednik'
elif bestIndex == 2:
    name = 'Sklad'
else:
    name = 'TC'

print(f'The best alternative is {name}, total costs for 5 years: {theBest}')